In [ ]:

import sys
sys.path.insert(0, r'F:\f1analytics\scripts')


import requests as rq

from converter import convert_ntt
import requests as rq
from sqlalchemy import create_engine

from multiprocessing import Process
from multiprocessing import Manager
import time

engine=create_engine('')
conn=engine.connect()
def getData(ttList):
    while True:
        ttList.append(rq.get(r'https://f1.tfeed.net/tt.js').text)
def toDB(ttList):
    text=ttList.pop(0)
    tlmy,wthr,race=convert_ntt(text)
    fullDf=tlmy.merge(wthr,on='timestamp').merge(race,on='timestamp')
    fullDf['session_name']='2024_australia'
    fullDf.to_sql('testing',conn,if_exists='append')


if __name__=='__main__':
    mgr = Manager()
    ttList=mgr.list()

    p1=Process(target=getData,args=(ttList,))
    p2=Process(target=toDB,args=(ttList,))
    p1.start()
    time.sleep(5)
    p2.start()
 

In [1]:

import sys
sys.path.insert(0, r'F:\f1analytics\scripts')
import os
import pandas as pd
from sqlalchemy import create_engine
engine=create_engine(os.environ.get('stbdb_url'))
conn=engine.connect()
import requests as rq
INFLUXDB_TOKEN=os.environ.get('influxdb_token')
from converter import convert_ntt

import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = INFLUXDB_TOKEN
org = "wardoge-analytics"
url = "192.168.1.66:8086"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
bucket="f1analytics"
write_api = write_client.write_api(write_options=SYNCHRONOUS)
while True:   
    tlmy_df,weather_df,race_df=convert_ntt(rq.get(r'https://f1.tfeed.net/tt.js').text.replace('\n','').replace('\t','').replace('\r','').replace('//',''),None)
    tlmy_df['timestamp'] = pd.to_datetime(tlmy_df['timestamp'],unit='ms')
    tlmy_df['throttle']=tlmy_df['throttle'].astype(int)
    tlmy_df=tlmy_df.set_index('timestamp')
    tlmy_df.to_sql('telemetry_test_2',conn,if_exists='append')
    #weather_df.to_sql('weather_df',conn,if_exists='append')
    #race_df.to_sql('race_test',conn,if_exists='append')
    #write_api.write(bucket, record = tlmy_df,data_frame_measurement_name='telemetry')
    

In [11]:
import pandas as pd
import os
from sqlalchemy import create_engine
engine=create_engine(os.environ.get('stbdb_url'))
conn=engine.connect()
SQL_QUERY='''
SELECT * FROM telemetry_test_2 WHERE laps=12 ORDER BY driver_num,timestamp
'''

test=pd.read_sql(SQL_QUERY,conn)

In [25]:
tlmy_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20 entries, 2024-04-05 03:18:41.408000 to 2024-04-05 03:18:41.408000
Data columns (total 42 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   driver_num       20 non-null     int64  
 1   status           20 non-null     object 
 2   laps             20 non-null     object 
 3   laptime          20 non-null     object 
 4   position         20 non-null     object 
 5   gap_from_leader  20 non-null     object 
 6   interval         20 non-null     object 
 7   pits             20 non-null     object 
 8   is_best          20 non-null     object 
 9   speed            20 non-null     object 
 10  gear             20 non-null     object 
 11  gear_switches    20 non-null     object 
 12  drs              20 non-null     object 
 13  lap_pos          20 non-null     object 
 14  engine_rpm       20 non-null     object 
 15  tyreset          20 non-null     object 
 16  speedtraps  